In [1]:
import torch,tiktoken
from torch.utils.data import Dataset,DataLoader
class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)
        for i in range(0,len(token_ids) - max_length,stride):
            input_chunk  = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1 :i +max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, index):
        return self.input_ids[index],self.target_ids[index]

def create_dataloader_v1(txt, batch_size=4,
        max_length=256, stride=128, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)
    dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=drop_last)
    return dataloader

In [2]:
output_dim = 256
vocab_size = 50257
max_length = 4
token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)
with open("the-verdict.txt","r",encoding="utf-8") as f:
    raw_text = f.read();
dataloader = create_dataloader_v1(raw_text,batch_size=8,max_length=max_length,stride=max_length)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)
print(token_embeddings)


Token IDs:
 tensor([[  938,   339, 13055,    11],
        [  314,   836,   470, 45553],
        [   40,  1276,  1107,   766],
        [  466,  1194, 14000,    13],
        [  345,  1683, 45553,   903],
        [   11, 10371,   257,  1534],
        [  520,  5493,    82,  1302],
        [  286,   616,  4286,   705]])

Inputs shape:
 torch.Size([8, 4])
torch.Size([8, 4, 256])
tensor([[[ 1.3183, -0.2563,  0.7627,  ..., -0.6953,  1.4740, -0.9624],
         [ 1.6142,  1.1780, -0.2096,  ...,  0.0241,  1.2200, -1.0252],
         [-1.1150,  0.2280,  1.1243,  ...,  0.2122,  1.2482, -0.1920],
         [ 1.9317,  0.6534,  0.4670,  ..., -2.0068, -1.5714, -1.1566]],

        [[ 0.9374,  0.3580,  2.9299,  ...,  2.6589, -0.0783, -0.3468],
         [ 1.0085,  0.1684, -2.8103,  ..., -0.4716,  0.1496,  1.0750],
         [ 0.0443,  0.9462,  0.5753,  ..., -1.1567,  0.1821, -0.1174],
         [-0.0158,  0.5259,  2.2099,  ...,  1.1880,  0.2722,  0.0843]],

        [[ 0.3209,  1.4145, -0.6278,  ..., -0.2335, 

In [3]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([4, 256])
torch.Size([8, 4, 256])
